In [1]:
import numpy as np
import pylab as pl
import math as ma
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import subprocess
import os
%matplotlib inline
from mpl_toolkits.axes_grid1.inset_locator import mark_inset,inset_axes,InsetPosition

In [12]:
#load lightcurves
path_dir='/path/to/data/files/'
bands='upperBand_lowerBand'

#data file format is mjd, flux, fluxerr
data1=np.loadtxt(path_dir+'higher_frequency_lc.txt')#higher frequency
data2=np.loadtxt(path_dir+'lower_frequency_lc.txt')#lower frequency

day_1=data1[:,0]
flux_1=data1[:,1]
fluxerr_1=data1[:,2]
day_2=data2[:,0]
flux_2=data2[:,1]
fluxerr_2=data2[:,2]


In [ ]:
#plot lightcurves
fig=plt.figure()
ax=plt.subplot(211)
ax.errorbar(day_1,flux_1,yerr=fluxerr_1,marker='o',ls='',color='b')
ax2=plt.subplot(212,sharex=ax)
ax2.errorbar(day_2,flux_2,yerr=fluxerr_2,marker='o',ls='',color='r')

##*Get ZDCF Fortran code [here](http://www.weizmann.ac.il/particle/tal/research-activities/software)*
###I modified original code to use the following parameters as commmand line arguments rather than user input:
* type (1=auto-correlation, 2=cross-correlation)
* output files prefix 
* are lightcurves uniformally sampled? (y or n)
* minimum points per bin
* omit zero lag when calculating ccf? (y or n)
* monte carlo simulations number for errors
* input light curve file(s)

In [14]:
#run zdcf code and ML code using subprocess module

# range in seconds to look for lags
lowr,upr=['-5000','5000']

#set up lightcurve files in proper format
os.system('rm -rf '+path_dir+'inp1')
os.system('rm -rf '+path_dir+'inp1')
os.system('rm -rf '+path_dir+'zdcf_light.dcf')
os.system('rm -rf '+path_dir+'zdcf_light.lc1')
os.system('rm -rf '+path_dir+'zdcf_light.lc2')
tempFile = open(path_dir+'inp1','w')
tempFile2 = open(path_dir+'inp2','w')
if day_1[0]<=day_2[0]:
    comp=day_1[0]
else:
    comp=day_2[0]
for i in range(0,(len(day_1)-1)):
    tempFile.write(' {0} {1} {2}\n'.format((day_1[i]-comp)*(60*60*24.),flux_1[i],fluxerr_1[i]))
for i in range(0,(len(day_2)-1)):
    tempFile2.write(' {0} {1} {2}\n'.format((day_2[i]-comp)*(60*60*24.),flux_2[i],fluxerr_2[i]))
tempFile.close()
tempFile2.close()

#compiled fortran code run
print 'Running ZDCF...'
'''subprocess arguments:
compiled code path,type (1=auto, 2=cross),output files prefix, 
uniform sampling? (y or n),min points per bin, omit zero lag? (y or n),
mc sim number for errors, input lc files'''
rc=subprocess.call(['/path/to/zdcf_code_compiled','2', path_dir+'zdcf_light', 'n',\
                    '11', 'y', '100', path_dir+'inp1', path_dir+'inp2'])
print 'Running ML Peak Finder...'
rc2=subprocess.call(['/path/to/zdcf_ml_compiled_code',path_dir+'zdcf_light.dcf',\
                     lowr,upr])
rc3=subprocess.check_output(['/path/to/zdcf_ml_compiled_code/zdcf_ml_cmpiled_code',\
                             path_dir+'zdcf_light.dcf',  lowr,upr])
print rc3

Running ZDCF...
Running ML Peak Finder...
 PLIKE V3.0 begins.
 Calculating ML in range   -5000.00000      to    5000.00000    

ZDCF peak at   +240.     r_max =  +0.422     , ZDCF C.O.M at   +214.    
....................


  #      lag         r         -dr        +dr    likelihood 
 --- ---------- ---------- ---------- ---------- ---------- 
  1  -540.     -0.641      0.267      0.412      2.525E-03
  2  -292.      0.166      0.339      0.318      2.010E-02
  3  -240.      0.186      0.337      0.315      2.249E-02
  4  -210.      7.886E-02  0.323      0.314      1.009E-02
  5  -180.      5.876E-02  0.307      0.301      7.374E-03
  6  -150.      0.106      0.294      0.284      8.787E-03
  7  -120.      0.153      0.280      0.267      1.081E-02
  8  -90.0     -2.842E-02  0.269      0.271      2.083E-03
  9  -60.0     -0.303      0.227      0.247      7.451E-05
 10  -30.0     -0.333      0.208      0.227      2.352E-05
 11   30.0     -2.815E-02  0.244      0.245      1.193E-03
 12  

In [26]:
#manually record lag output, all in seconds
lag_val=240.
lag_errL=-256.
lag_errH=415.

##plot limits
#inset around peak
in_low=-500
in_up=500
#full plot
low=-1000
up=1000

In [ ]:
#plot ccf
data=np.loadtxt(path_dir+'zdcf_light.dcf')
time_lag=data[:,0]
time_lag_low=data[:,1]
time_lag_up=data[:,2]
cross_cor_coef=data[:,3]
cross_cor_coef_low=data[:,4]
cross_cor_coef_up=data[:,5]
filez=open(path_dir+'datafile_zdcf_'+bands+'.txt','w')
for ii in range(0,len(time_lag)):
    filez.write('{0} {1} {2} {3} {4} {5}\n'.format(time_lag[ii],time_lag_low[ii],time_lag_up[ii],\
                                                   cross_cor_coef[ii],cross_cor_coef_low[ii],\
                                                   cross_cor_coef_up[ii]))
filez.close()

fig2=plt.figure(figsize=(10,7))
from matplotlib import rc
rc('text', usetex=True)
font = {'family' : 'serif',
        'weight' : 'bold',
        'size'   : '14'}
rc('font', **font)
ax=plt.subplot(111)
ax.errorbar(time_lag/60.,cross_cor_coef,yerr=[cross_cor_coef_low,cross_cor_coef_up],\
            xerr=[time_lag_low/60.,time_lag_up/60.],marker='o',color='#58ACFA')
axins = inset_axes(ax,width="20%",height=2.,loc=2)
axins.errorbar(time_lag/60.,cross_cor_coef,yerr=[cross_cor_coef_low,cross_cor_coef_up],\
               xerr=[time_lag_low/60.,time_lag_up/60.],marker='o',color='#58ACFA')
axins.set_xlim(in_low/60.,in_up/60.)
axins.set_ylim(-0.6,0.6)
axins.set_xticks([0,3,6])
axins.set_yticks([-0.6,-0.3,0,0.3,0.6])
axins.axvline(x=lag_val/60.,color='k',lw=1.5,ls='--')
axins.axvspan(lag_errL/60., lag_errH/60., alpha=0.15, color='k')
axins.tick_params(axis='both',which='minor',length=4,width=1)
axins.tick_params(axis='both',which='major',labelsize=28,length=6,width=1)
axins.xaxis.set_minor_locator(AutoMinorLocator(3))
ip = InsetPosition(ax,[0.16,0.66,0.3,0.3])
axins.set_axes_locator(ip)
ax.set_xlabel('Time Lag (minutes)',fontsize=28)
ax.set_ylabel('Correlation Coefficient',fontsize=28)
ax.tick_params(axis='both',which='minor',length=4,width=1)
ax.tick_params(axis='both',which='major',labelsize=28,length=6,width=1)
ax.set_xlim(low/60.,up/60.)
ax.set_ylim(-1,1.)
ax.axvline(x=lag_val/60.,color='k',lw=1.5,ls='--')
ax.axvspan(lag_errL/60., lag_errH/60., alpha=0.15, color='k')
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.yaxis.set_minor_locator(AutoMinorLocator(2))
plt.savefig(path_dir+'zdcf_'+bands+'_fig.pdf',format='pdf',bbox_inches='tight')